In [10]:
!pip install pathlib
!python -m pip install -U ultralytics

In [20]:
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import cv2
import torch
import shutil
from pathlib import Path
from ultralytics import YOLO


warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/segmentation-data/labels_names_220425.npy
/kaggle/input/segmentation-data/videos_1.npy
/kaggle/input/segmentation-data/videos.npy
/kaggle/input/segmentation-data/masks_1.npy
/kaggle/input/segmentation-data/videos_check_220425.npy
/kaggle/input/segmentation-data/labels_220425.npy
/kaggle/input/segmentation-data/masks.npy
/kaggle/input/segmentation-data/masks_check_220425.npy
/kaggle/input/segmentation-data/yolo_seg_small_040325.pt


# Подключение датасета и загрузка данных

In [12]:
# Проверка доступности CUDA
if torch.cuda.is_available():
    print("CUDA доступна")
    print("Имя устройства:", torch.cuda.get_device_name(0))
    print("Кол-во устройств:", torch.cuda.device_count(), "\n")

# Подключаем датасет
project_dir = '/kaggle/input/segmentation-data/'
videos_file = os.path.join(project_dir, 'videos_1.npy')
masks_file = os.path.join(project_dir, 'masks_1.npy')

try:
    videos = np.load(videos_file)
    print("Файл 'videos' загружен успешно.\n")
except FileNotFoundError:
    print("Файл 'videos' не найден. Проверьте правильность пути.\n")

videos = np.load(videos_file)
masks = np.load(masks_file)

print(f"Размерность массива видео: {videos.shape}")     # (Число видео, Число кадров, Высота, Ширина)
print(f"Размерность массива меток: {masks.shape}")      # (Размерности должны совпадать)

if videos.shape == masks.shape:
    print("Размерности 'videos' и 'masks' совпадают")
else:
    print("Размерности 'videos' и 'masks' не совпадают")

print(type(videos))

Файл 'videos' загружен успешно.

Размерность массива видео: (412, 54, 224, 224)
Размерность массива меток: (412, 54, 224, 224)
Размерности 'videos' и 'masks' совпадают
<class 'numpy.ndarray'>


# выберем случайным образом 50 видео

In [13]:
num_to_save = 50

selected_indices = np.random.choice(videos.shape[0], num_to_save, replace=False)
remaining_indices = np.setdiff1d(np.arange(videos.shape[0]), selected_indices)

videos_saved = videos[selected_indices]
masks_saved = masks[selected_indices]

images_saved = videos_saved.reshape(-1, 224, 224)
masks_saved_images = masks_saved.reshape(-1, 224, 224)

videos = videos[remaining_indices]
masks = masks[remaining_indices]

print('размеры массивов до превращения видео в набор картинок')
print(videos.shape)
print(masks.shape)

размеры массивов до превращения видео в набор картинок
(362, 54, 224, 224)
(362, 54, 224, 224)


# Инициализация модели и функция создания маски по видео

In [22]:
model = os.path.join(project_dir, 'yolo_seg_small_040325.pt')
model = YOLO(model)

def process_and_save_video_all(video_array: np.ndarray,
                               model,
                               output_original: str,
                               output_detection: str,
                               output_segmentation: str,
                               conf_threshold: float = 0.5,
                               mask_threshold: float = 0.5,
                               fps: int = 10,
                               detection_color: tuple = (0, 255, 0),
                               mask_color: tuple = (255, 255, 255)):
    """
    Обрабатывает видео и сохраняет три файла:
      1) оригинальное видео,
      2) видео с детекцией bbox,
      3) видео с бинарными масками сегментации.

    :param video_array: np.ndarray формы (N, H, W) или (N, H, W, 3)
    :param model: YOLOv11_seg-модель (Ultralytics), поддерживает task="segment"
    :param output_original: путь для сохранения оригинального видео (.mp4)
    :param output_detection: путь для видео с детекцией bbox
    :param output_segmentation: путь для видео с сегментацией (чёрно-белые кадры)
    :param conf_threshold: минимальный confidence для боксов
    :param mask_threshold: порог бинаризации вероятностных масок
    :param fps: кадры в секунду
    :param detection_color: цвет bbox (B, G, R)
    :param mask_color: цвет объекта в маске (по умолчанию белый)
    """
    N, H, W = video_array.shape[0], video_array.shape[1], video_array.shape[2]

    # подготовка
    def make_writer(path):
        p = Path(path)
        p.parent.mkdir(parents=True, exist_ok=True)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        return cv2.VideoWriter(str(p), fourcc, fps, (W, H))
    
    writer_orig = make_writer(output_original)
    writer_det  = make_writer(output_detection)
    writer_seg  = make_writer(output_segmentation)

    for i in range(N):
        frame = video_array[i]
        # приводим к BGR uint8
        if frame.ndim == 2:
            frame_bgr = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
        else:
            frame_bgr = frame.astype(np.uint8)

        # сохраняем оригинал
        writer_orig.write(frame_bgr)

        # предсказание
        results = model(frame_bgr, conf=conf_threshold, task="segment", verbose=False)[0]

        #видео с детекцией
        det_frame = frame_bgr.copy()
        for box in results.boxes.data.cpu().numpy():
            x1, y1, x2, y2, conf, cls = box
            if conf < conf_threshold:
                continue
            pt1 = (int(x1), int(y1))
            pt2 = (int(x2), int(y2))
            cv2.rectangle(det_frame, pt1, pt2, detection_color, 2)
            label = f"{results.names[int(cls)]} {conf:.2f}"
            cv2.putText(det_frame, label, (pt1[0], pt1[1] - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, detection_color, 2)
        writer_det.write(det_frame)

        #видео с сегментацией
        mask_output = np.zeros((H, W), dtype=np.uint8)
        if results.masks is not None and results.masks.data.numel() > 0:
            masks = results.masks.data.cpu().numpy()  # shape: (M, H, W)
            for mask in masks:
                binary = (mask > mask_threshold).astype(np.uint8) * mask_color[0]  # маска белая
                mask_output = np.maximum(mask_output, binary)
        mask_bgr = cv2.cvtColor(mask_output, cv2.COLOR_GRAY2BGR)
        writer_seg.write(mask_bgr)

    writer_orig.release()
    writer_det.release()
    writer_seg.release()

    print("Сохранены:")
    print(f"   • Оригинал:   {output_original}")
    print(f"   • Детекция:   {output_detection}")
    print(f"   • Сегментации:{output_segmentation}")

In [23]:
# Пример вызова:
num_video = 30

output_path=f'/kaggle/working/final_videos/'

process_and_save_video_all(
    video_array=videos_saved[num_video],
    model=model,
    output_original = output_path + f"video{num_video}.mp4",
    output_detection = output_path + f"video{num_video}_det.mp4",
    output_segmentation = output_path + f"video{num_video}_seg.mp4",
    conf_threshold=0.5,
    mask_threshold=0.5,
    fps=15
)

Сохранены:
   • Оригинал:   /kaggle/working/final_videos/video30.mp4
   • Детекция:   /kaggle/working/final_videos/video30_det.mp4
   • Сегментации:/kaggle/working/final_videos/video30_seg.mp4
